In [18]:
# import required libriaries
import requests
import pandas as pd
import numpy as np
import lxml.html as lh
from pathlib import Path

In [19]:
# path to data directory
path = Path('/home/jupyter/fpl-prediction/data')

In [28]:
# paths to each season's data
season_paths = [path/'2016-17', path/'2017-18', path/'2018-19', path/'2019-20']

# names for each season
season_names = ['1617', '1718', '1819', '1920']

# read in player information for each season and add to list
season_players = []

for season_path in season_paths:
    players = pd.read_csv(season_path/'players_raw.csv', 
                           usecols=['first_name', 'second_name', 'id', 
                                    'team_code', 'element_type', 'now_cost',
                                    'chance_of_playing_next_round'])
    season_players.append(players)
    
# read in team information (all seasons)
teams = pd.read_csv(path/'teams.csv')

# two danny wards in 1819, rename the new one
season_players[2].loc[143, 'second_name'] = 'Ward_2'

# create full name field for each player
for players in season_players:
    players['full_name'] = players['first_name'] + '_' + players['second_name']
    players.drop(['first_name', 'second_name'], axis=1, inplace=True)

# create series of all unique player names
all_players = pd.concat(season_players, axis=0, ignore_index=True, sort=False)
all_players = pd.DataFrame(all_players['full_name'].drop_duplicates())

# create player dataset with their id, team code and position id for each season
for players, season in zip(season_players, season_names):
    all_players = all_players.merge(players, on='full_name', how='left')
    all_players.rename(index=str,
                       columns={'id':'id_' + season,
                                'team_code':'team_' + season,
                                'element_type': 'position_' + season,
                                'now_cost': 'cost_' + season,
                                'chance_of_playing_next_round': 'play_proba_' + season},
                       inplace=True)

In [30]:
# players dataset
all_players.head(50)

,full_name,play_proba_1617,position_1617,id_1617,cost_1617,team_1617,play_proba_1718,position_1718,id_1718,cost_1718,...,play_proba_1819,position_1819,id_1819,cost_1819,team_1819,play_proba_1920,position_1920,id_1920,cost_1920,team_1920
0,David_Ospina,100,1.0,1.0,47.0,3.0,100,1.0,1.0,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Petr_Cech,100,1.0,2.0,54.0,3.0,100,1.0,2.0,53.0,...,100,1.0,1.0,47.0,3.0,NaN,NaN,NaN,NaN,NaN
2,Laurent_Koscielny,0,2.0,3.0,61.0,3.0,0,2.0,4.0,60.0,...,100,2.0,3.0,54.0,3.0,0,2.0,7.0,50.0,3.0
3,Per_Mertesacker,100,2.0,4.0,48.0,3.0,100,2.0,5.0,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gabriel Armando_de Abreu,75,2.0,5.0,48.0,3.0,0,2.0,6.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Héctor_Bellerín,100,2.0,6.0,65.0,3.0,100,2.0,7.0,59.0,...,0,2.0,4.0,53.0,3.0,0,2.0,2.0,55.0,3.0
6,Kieran_Gibbs,75,2.0,7.0,47.0,3.0,100,2.0,8.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Mathieu_Debuchy,100,2.0,8.0,46.0,3.0,0,2.0,564.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Carl_Jenkinson,100,2.0,9.0,48.0,3.0,0,2.0,9.0,45.0,...,None,2.0,581.0,45.0,3.0,0,2.0,9.0,45.0,3.0
9,Nacho_Monreal,100,2.0,10.0,59.0,3.0,100,2.0,10.0,57.0,...,100,2.0,5.0,54.0,3.0,None,2.0,6.0,50.0,3.0


In [31]:
# check columns are pandas series
type(all_players.loc[:,'id_1617'])

pandas.core.series.Series

In [66]:
all_players[all_players['team_1920'] == 14]

,full_name,play_proba_1617,position_1617,id_1617,cost_1617,team_1617,play_proba_1718,position_1718,id_1718,cost_1718,...,play_proba_1819,position_1819,id_1819,cost_1819,team_1819,play_proba_1920,position_1920,id_1920,cost_1920,team_1920
17,Alex_Oxlade-Chamberlain,100,3.0,18.0,58.0,3.0,0,3.0,19.0,55.0,...,100,3.0,248.0,64.0,14.0,None,3.0,193.0,65.0,14.0
151,Andrew_Robertson,100,2.0,152.0,44.0,88.0,100,2.0,475.0,50.0,...,100,2.0,247.0,67.0,14.0,None,2.0,181.0,70.0,14.0
187,Simon_Mignolet,None,1.0,188.0,48.0,14.0,None,1.0,213.0,47.0,...,100,1.0,237.0,47.0,14.0,0,1.0,190.0,45.0,14.0
190,Dejan_Lovren,100,2.0,191.0,49.0,14.0,100,2.0,215.0,53.0,...,100,2.0,239.0,49.0,14.0,None,2.0,186.0,55.0,14.0
194,Joseph_Gomez,100,2.0,195.0,48.0,14.0,0,2.0,218.0,45.0,...,100,2.0,242.0,48.0,14.0,None,2.0,184.0,55.0,14.0
198,Joel_Matip,100,2.0,199.0,54.0,14.0,0,2.0,219.0,54.0,...,100,2.0,243.0,49.0,14.0,None,2.0,185.0,55.0,14.0
200,Jordan_Henderson,75,3.0,201.0,62.0,14.0,100,3.0,225.0,53.0,...,100,3.0,249.0,53.0,14.0,None,3.0,198.0,55.0,14.0
204,Adam_Lallana,100,3.0,205.0,67.0,14.0,100,3.0,228.0,73.0,...,100,3.0,250.0,69.0,14.0,None,3.0,195.0,60.0,14.0
207,James_Milner,100,3.0,208.0,63.0,14.0,100,2.0,222.0,60.0,...,100,3.0,254.0,56.0,14.0,100,3.0,200.0,55.0,14.0
208,Roberto_Firmino,100,3.0,209.0,85.0,14.0,100,4.0,235.0,95.0,...,100,4.0,257.0,93.0,14.0,None,4.0,187.0,95.0,14.0


In [33]:
all_players.shape

(1190, 21)

In [34]:
# teams dataset
teams

,team,team_code,team_1617,team_1718,team_1819,team_1920
0,Arsenal,3,1.0,1.0,1.0,1.0
1,Bournemouth,91,2.0,2.0,2.0,3.0
2,Burnley,90,3.0,4.0,4.0,5.0
3,Chelsea,8,4.0,5.0,6.0,6.0
4,Crystal Palace,31,5.0,6.0,7.0,7.0
5,Everton,11,6.0,7.0,8.0,8.0
6,Hull City,88,7.0,NaN,NaN,NaN
7,Leicester City,13,8.0,9.0,11.0,9.0
8,Liverpool,14,9.0,10.0,12.0,10.0
9,Manchester City,43,10.0,11.0,13.0,11.0


In [35]:
# function to scrape market value of premier league teams at the start of each season
# from www.transfermarkt.com
def build_season_mv(season, header_row, team_rows):
    
    # url for page with team market value at start of season
    url=r'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=' + '20' + season[0:2]
    
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')
    
    #Create empty list
    col=[]
    i=0

    #For each row, store each first element (header) and an empty list
    for t in tr_elements[header_row]:
        i+=1
        name=t.text_content()
        col.append((name,[]))
        
    #data is stored on the second row onwards
    for j in team_rows:
        #T is our j'th row
        T=tr_elements[j]

        #If row is not of size 10, the //tr data is not from our table 
        if len(T)!=10:
            break

        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1
        
    # create market values dataframe
    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    
    # convert market value string to float for millions of euros
    values = [float(item[0].replace(',', '.')) for item in df['Total MV'].str.split(" ", 1)]
    values = [item*10**3 if item < 3 else item for item in values]
    
    # to remove effect of inflation, take relative market value for each season
    values = values/np.mean(values)
    
    # create smaller dataframe with team names, market value and the season
    df = df[['name']]
    df.replace(team_names, inplace=True)
    df['relative_market_value'] = values
    df['season'] = season
    
    return df

In [36]:
# market value website has shortened team names
# lookup dictionary of full names
team_names = {'Man City': 'Manchester City',
              'Spurs': 'Tottenham Hotspur',
              'Man Utd': 'Manchester United',
              'Leicester': 'Leicester City',
              'West Ham': 'West Ham United',
              'Wolves': 'Wolverhampton Wanderers',
              'Brighton': 'Brighton and Hove Albion',
              'Newcastle': 'Newcastle United',
              'Sheffield Utd.': 'Sheffield United',
              'West Brom': 'West Bromwich Albion',
              'Swansea': 'Swansea City',
              'Huddersfield': 'Huddersfield Town',
              'Cardiff': 'Cardiff City'}

In [37]:
# create market value dataset for each season and combine
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = pd.concat([build_season_mv('1617', 10, range(12,32)),
                      build_season_mv('1718', 10, range(12,32)),
                      build_season_mv('1819', 10, range(12,32)),
                      build_season_mv('1920', 11, range(13,33))], 
                     axis=0, ignore_index=True, sort=False)

In [38]:
# check a season
teams_mv[teams_mv['season'] == '1819']

,name,relative_market_value,season
40,Manchester City,2.540586,1819
41,Chelsea,2.540586,1819
42,Liverpool,2.177887,1819
43,Manchester United,2.015531,1819
44,Tottenham Hotspur,1.983109,1819
45,Arsenal,1.435552,1819
46,Everton,1.039221,1819
47,Leicester City,0.796050,1819
48,West Ham United,0.749352,1819
49,Southampton,0.654624,1819


In [39]:
# function to create season training dataset
# each player has a row for each gameweek
def build_season(path, season):
    
    # season specific list and strings to use for merging
    df_season = []
    id_season = 'id_' + season
    id_team = 'team_' + season
    id_position = 'position_' + season
    
    # read in each gameweek and append to season list
    for i in range(1, 39):
        gw = 'gws/gw' + str(i) + '.csv'
        gw_df = pd.read_csv(path/gw, encoding='latin')
        gw_df['gw'] = i
        df_season.append(gw_df)
    
    # concatenate entire season
    df_season = pd.concat(df_season, axis=0)
    
    # join to player, team and team market value datasets to create season training set
    df_season = df_season.merge(all_players, left_on='element', right_on=id_season, how='left')
    df_season = df_season.merge(teams, left_on='opponent_team', right_on=id_team, how='left')
    df_season = df_season.merge(teams, left_on=id_team + '_x', right_on='team_code', how='left')
    df_season = df_season.merge(teams_mv[teams_mv['season'] == season], 
                                left_on='team_x', right_on='name', how='left')
    df_season = df_season.merge(teams_mv[teams_mv['season'] == season], 
                                left_on='team_y', right_on='name', how='left')
    df_season = df_season[['full_name', 'gw', 
                           id_position, 'minutes', 'team_y', 
                           'team_x', 'relative_market_value_y', 
                           'relative_market_value_x', 'was_home', 'total_points']]
    df_season.columns = ['player', 'gw', 
                          'position', 'minutes', 'team', 
                          'opponent_team', 'relative_market_value_team', 
                          'relative_market_value_opponent_team', 'was_home', 'total_points']
    df_season['season'] = season
    df_season['position'] = df_season['position'].astype(int)
    
    return df_season

In [40]:
# create training sets for each season
df_1617 = build_season(season_paths[0], season_names[0])
df_1718 = build_season(season_paths[1], season_names[1])
df_1819 = build_season(season_paths[2], season_names[2])

In [41]:
# size of each season
[x.shape for x in [df_1617, df_1718, df_1819]]

[(23679, 11), (22467, 11), (21790, 11)]

In [42]:
# join together into one dataframe
df_train = pd.concat([df_1617, df_1718, df_1819], ignore_index=True, axis=0)
#df_all = df_1617.append(df_1718).append(df_1819).reset_index(drop=True)

In [43]:
df_train.shape

(67936, 11)

In [44]:
df_train.tail(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season
67926,Yerry_Mina,38,2,90,Everton,Tottenham Hotspur,1.039221,1.983109,False,1,1819
67927,Yohan_Benalouane,38,2,0,Leicester City,Chelsea,0.796050,2.540586,True,0,1819
67928,Yoshinori_Muto,38,4,24,Newcastle United,Fulham,0.483921,0.405284,False,1,1819
67929,Younes_Kaboul,38,2,0,Watford,West Ham United,0.412301,0.749352,True,0,1819
67930,Youri_Tielemans,38,3,90,Leicester City,Chelsea,0.796050,2.540586,True,3,1819
67931,Yves_Bissouma,38,3,90,Brighton and Hove Albion,Manchester City,0.417745,2.540586,True,2,1819
67932,Zech_Medley,38,2,0,Arsenal,Burnley,1.435552,0.449442,False,0,1819
67933,Zeze Steven_Sessegnon,38,2,0,Fulham,Newcastle United,0.405284,0.483921,True,0,1819
67934,Álvaro_Morata,38,4,0,Chelsea,Leicester City,2.540586,0.796050,False,0,1819
67935,Çaglar_Söyüncü,38,2,0,Leicester City,Chelsea,0.796050,2.540586,True,0,1819


In [45]:
# save latest training set to csv
df_train.to_csv(path/'train.csv')#, encoding='latin')

In [46]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [47]:
fixtures.head(10)

,home_team,away_team,gw
0,Liverpool,Norwich,1
1,West Ham United,Manchester City,1
2,Bournemouth,Sheffield United,1
3,Burnley,Southampton,1
4,Crystal Palace,Everton,1
5,Watford,Brighton and Hove Albion,1
6,Tottenham Hotspur,Aston Villa,1
7,Leicester City,Wolverhampton Wanderers,1
8,Newcastle United,Arsenal,1
9,Manchester United,Chelsea,1


In [48]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [49]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,1,Liverpool,Norwich,14,45
1,1,West Ham United,Manchester City,21,43
2,1,Bournemouth,Sheffield United,91,49
3,1,Burnley,Southampton,90,20
4,1,Crystal Palace,Everton,31,11


In [50]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [51]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [52]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,1,Norwich,Liverpool,Sam_Byram,2.0,45.0,None,False
1,1,Norwich,Liverpool,Tim_Krul,1.0,45.0,None,False
2,1,Norwich,Liverpool,Grant_Hanley,2.0,40.0,None,False
3,1,Norwich,Liverpool,Timm_Klose,2.0,45.0,0,False
4,1,Norwich,Liverpool,Ben_Godfrey,2.0,45.0,100,False


In [53]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,1,Liverpool,Norwich,Alex_Oxlade-Chamberlain,3.0,65.0,None,True
1,1,Liverpool,Norwich,Andrew_Robertson,2.0,70.0,None,True
2,1,Liverpool,Norwich,Simon_Mignolet,1.0,45.0,0,True
3,1,Liverpool,Norwich,Dejan_Lovren,2.0,55.0,None,True
4,1,Liverpool,Norwich,Joseph_Gomez,2.0,55.0,None,True


In [58]:
home_df.loc[home_df['play_proba'] == 'None', 'play_proba'] = 100

0       None
1       None
3       None
4       None
5       None
6       None
7       None
9       None
10      None
11      None
12      None
14      None
15      None
16      None
17      None
19      None
20      None
21      None
22      None
23      None
24      None
27      None
28      None
29      None
32      None
33      None
34      None
35      None
36      None
37      None
        ... 
9672    None
9673    None
9674    None
9676    None
9677    None
9679    None
9680    None
9681    None
9682    None
9683    None
9686    None
9687    None
9688    None
9691    None
9692    None
9693    None
9694    None
9695    None
9696    None
9698    None
9699    None
9700    None
9701    None
9702    None
9703    None
9704    None
9705    None
9706    None
9707    None
9708    None
Name: play_proba, Length: 7695, dtype: object

In [72]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# set minutes equal to 90 multiplied by their play probabiliby for all players, for now
remaining_season_df['minutes'] = 90 * remaining_season_df['play_proba']

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

In [76]:
remaining_season_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,1,Liverpool,Norwich,Alex_Oxlade-Chamberlain,3,6.5,1.0,True,2.291459,0.197772,1920,90.0
1,1,Liverpool,Norwich,Andrew_Robertson,2,7.0,1.0,True,2.291459,0.197772,1920,90.0
2,1,Liverpool,Norwich,Simon_Mignolet,1,4.5,0.0,True,2.291459,0.197772,1920,0.0
3,1,Liverpool,Norwich,Dejan_Lovren,2,5.5,1.0,True,2.291459,0.197772,1920,90.0
4,1,Liverpool,Norwich,Joseph_Gomez,2,5.5,1.0,True,2.291459,0.197772,1920,90.0


In [77]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')